In [4]:
%%time
print("start")
%run -i 'myimports.py'

def coordinates(title):
    return [40, 220, 150, 300, 183, 494] #first last pages, x coord and width, y coord and height

def create_txt(title):
    if not os.path.isfile(title+".txt"):
        if os.path.isfile(title+".pdf"):
            c = coordinates(title+".pdf")
            convert = "pdftotext -f %d -l %d -x %d -W %d -y %d -H %d %s.pdf" %(c[0],c[1],c[2],c[3],c[4],c[5],title)
            process = subprocess.Popen(convert.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()

contents = ["Introduction",
            "The Negro and Language",
            "The Woman of Color and the White Man",
            "The Man of Color and the White Woman",
            "The So-Called Dependency Complex of Colonized",
            "Peoples",
            "The Fact of Blackness",
            "The Negro and Psychopathology",
            "The Negro and Recognition",
            "By Way of Conclusion"]
contents = [x.upper() for x in contents]

start
CPU times: user 1.61 ms, sys: 657 µs, total: 2.27 ms
Wall time: 3.63 ms


In [5]:
# split_text = full_text.split("")

full_text = ""
with open("black.txt", "r") as r:
    lines = r.readlines()
    for line in lines:
        full_text +=line.replace("\n", " ")


# os.mkdir(os.getcwd() +"/chapters", 0o755)

for i in range(0, len(contents)):
    t = full_text.split(contents[i])
    print(i, contents[i])
    with open("chapters/"+str(i-1)+"_"+contents[i-1].replace(" ", "_")+".txt", "w+") as f:
        f.write(t[0])
    full_text = t[1]
# with open("chapters2"+str(len(contents)-1)+" "+contents[-1]+".txt", "w") as f:
#     f.write(file1.replace(" ", "_"))

0 INTRODUCTION
1 THE NEGRO AND LANGUAGE
2 THE WOMAN OF COLOR AND THE WHITE MAN
3 THE MAN OF COLOR AND THE WHITE WOMAN
4 THE SO-CALLED DEPENDENCY COMPLEX OF COLONIZED
5 PEOPLES
6 THE FACT OF BLACKNESS
7 THE NEGRO AND PSYCHOPATHOLOGY
8 THE NEGRO AND RECOGNITION
9 BY WAY OF CONCLUSION
